In [13]:
!pip install diffusers transformers scipy torch streamlit pyngrok


In [14]:
from diffusers import StableDiffusionInpaintPipeline
import torch
from PIL import Image, ImageDraw
from google.colab import files
import io
from pyngrok import ngrok


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [15]:
model_id = "runwayml/stable-diffusion-inpainting"
pipe = StableDiffusionInpaintPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")  # Ensure you're using GPU for faster processing


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

safety_checker/model.safetensors not found


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

diffusion_pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_

In [16]:
# Authenticate with ngrok
!ngrok authtoken 2niY3s9EQG0hV41VJDpSamEn3nA_2yCPJZdjJTZnryyqukqTt

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [24]:
%%writefile app.py
import streamlit as st
from PIL import Image, ImageDraw
import torch
from diffusers import StableDiffusionInpaintPipeline
import io

# Load the Stable Diffusion inpainting model
model_id = "runwayml/stable-diffusion-inpainting"
pipe = StableDiffusionInpaintPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# Streamlit app layout
st.title("INTERIOR AI")

# Upload the image
uploaded_file = st.file_uploader("Upload a photo of the room", type=["jpg", "jpeg", "png"])
if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_column_width=True)

    # Get user input for design preferences
    description = st.text_input("Describe how you want the room to look (e.g., modern, minimalistic, cozy)")

    if st.button("Generate Design"):
        if description:
            # Generate a mask for the areas where we want to apply changes
            mask = Image.new("L", image.size, 0)
            draw = ImageDraw.Draw(mask)
            width, height = image.size
            draw.rectangle([0, int(height / 2), width, height], fill=255)  # Mask lower half

            # Generate room design
            prompt = f"A {description} room with furniture and decor."
            generated_image = pipe(prompt=prompt, image=image, mask_image=mask).images[0]
            st.image(generated_image, caption='Generated Room Design', use_column_width=True)
        else:
            st.warning("Please enter your design preferences.")


Overwriting app.py


In [27]:
# Correcting the ngrok connection setup
public_url = ngrok.connect(8501, "http")  # Specify the port and protocol (http)
print(f"Streamlit app will be available on: {public_url}")


Streamlit app will be available on: NgrokTunnel: "https://d267-34-125-12-173.ngrok-free.app" -> "http://localhost:8501"


In [19]:
!streamlit run app.py --server.port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8503
  Network URL: http://172.28.0.12:8503
  External URL: http://34.125.12.173:8503

  Stopping...
^C


In [ ]:
ngrok config add-authtoken $2niY3s9EQG0hV41VJDpSamEn3nA_2yCPJZdjJTZnryyqukqTt

In [20]:
pip install streamlit -q

In [21]:
!wget -q -O - ipv4.icanhazip.com

34.125.12.173


In [28]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.12.173:8501

your url is: https://quick-facts-count.loca.lt
2024-10-21 04:53:31.586389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 04:53:31.748934: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 04:53:31.794632: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 04:53:34.381225: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
safety_checker/mod

^C


In [12]:
!npm install -g localtunnel



added 22 packages, and audited 23 packages in 3s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
